# Imports

In [35]:
import pandas as pd

testdir    = '/Users/erichganz/Documents/Code/Kaggle/AmericanExpress/data/raw/train_labels.csv'

df         = pd.read_parquet('/Users/erichganz/Documents/Code/Kaggle/AmericanExpress/data/parquets/train.parquet', engine='pyarrow')
testLables = pd.read_csv(testdir)

testLables['ID'] = testLables.index.values

In [36]:
# Shorten cus_ids
assignment     = dict(zip(testLables.customer_ID.values, testLables.index.values))
df.customer_ID = df.customer_ID.apply(lambda x: assignment[x]).astype('int32')

# getting datetime
df.S_2 = pd.to_datetime(df.S_2)

# Analysis

### Changing default rate by length of time period

In [87]:
df.customer_ID.value_counts().value_counts()

13    386034
12     10623
10      6721
9       6411
8       6110
2       6098
11      5961
3       5778
6       5515
7       5198
1       5120
4       4673
5       4671
Name: customer_ID, dtype: int64

In [89]:
comp_to_timelength          = pd.DataFrame()
comp_to_timelength['ID']    = df.customer_ID.value_counts().index.values
comp_to_timelength['group'] = df.customer_ID.value_counts().values

for group in comp_to_timelength.group.unique():

    print(group, testLables.loc[testLables.ID.isin(comp_to_timelength.loc[comp_to_timelength.group == group, 'ID'].unique()), 'target'].mean())

13 0.231787873606988
12 0.38934387649439894
11 0.44673712464351617
10 0.4622823984526112
9 0.45016378100140386
8 0.4472995090016367
7 0.41843016544824935
6 0.3876699909338169
5 0.3926354099764504
4 0.4162208431414509
3 0.35860159224645205
2 0.3184650705149229
1 0.3357421875


### Concate

In [94]:
# FEATURE ENGINEERING FROM 
# https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created

all_cols     = [c for c in list(df.columns) if c not in ['customer_ID','S_2']]
cat_features = ["B_30", 'B_31', "B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]
num_features = [col for col in all_cols if col not in cat_features]

test_num_agg = df.groupby("customer_ID")[num_features].agg(['mean', 'std', 'min', 'max', 'last'])
test_num_agg.columns = ['_'.join(x) for x in test_num_agg.columns]

test_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
test_cat_agg.columns = ['_'.join(x) for x in test_cat_agg.columns]

df_test = pd.concat([test_num_agg, test_cat_agg], axis=1)
    
  